## Headings Incremental Preprocessing (Part 1)

Formatting unseen headings (new headings in the persistent zone that do not exist already in the formatted zone). In this first part, generate the manual curation file for unseen headings.

In [11]:
from database_settings import spark_utilities
from pyspark.sql.functions import col, lpad,concat_ws
import pandas as pd
import time
from database_settings import postgres_utilities
from data_formatter import utilities

In [12]:
# Get headings to work
headings = utilities.get_headings()
headings_filter = r"^("+ "|".join(headings) + ")" # filter out headings that aren't in the list

In [13]:
# Get the headings that already exist in the Formatted Zone

# Establish the connection with the database
engine = postgres_utilities.engine()

# Read the headings table
existing_headings = pd.read_sql_table('peru_exports_headings', engine)

Connected to Formatted Zone successfully!


In [14]:
existing_headings

,heading,description,mapped_to
0,0701100000,"Papas (patatas) para siembra, frescos o refrig...",None
1,0701900000,"Papas (patatas) excepto para siembra, frescos ...",None
2,0702000000,Tomates frescos o refrigerados,None
3,0703100000,Cebollas y chalotes frescos o refrigerados,None
4,0703201000,Ajos para siembra frescos o refrigerados,None
...,...,...,...
169,0813300000,Manzanas secas,None
170,0813400000,Otros frutas o frutos secos,None
171,0813500000,Mezcla de frutas o frutos secos,None
172,0814001000,"Corteza de (limón sutil, limón común, limón cr...",None


In [15]:
# Get the headings in the Persistent Zone that do not exist in the Formatted Zone (those starting with the digits in the headings list defined above)
headings_to_format = spark_utilities.get_spark_df('peru_exports')\
    .select('PART_NANDI')\
    .distinct()\
    .withColumn("heading", lpad(col("PART_NANDI").cast("string"), 10, "0")) \
    .filter(col("heading").rlike(headings_filter))\
    .filter(~(col("heading").isin(existing_headings['heading'].to_list())))\
    .select('heading')\
    .toPandas()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [4]:

# Get the labeled headings from the persistent zone
headings_labeled = spark_utilities.get_spark_df('peru_exports_headings')\
    .select('0', '1')\
    .distinct()\
    .withColumnRenamed('1', 'raw_description') \
    .withColumn("heading", lpad(col("0").cast("string"), 10, "0")) \
    .select('heading', 'raw_description')\
    .toPandas()

# Join both dataframes in the headings column
my_headings = pd.merge(headings_to_format, headings_labeled, on='heading',how='left').sort_values(by='heading', ascending=True)

# Formatting
# Eliminate multiple spacings in the description
my_headings['raw_description'].replace(r'\s+', ' ', regex=True, inplace=True)
# Eliminate the dashes at the beginning of the description
my_headings['raw_description'].replace(r'^[-\s]+', '', regex=True, inplace=True)
# Add final dot to description
my_headings['raw_description'] = my_headings['raw_description'].apply(lambda x: x if x.endswith('.') else x+'.')
# Group all descriptions from a heading in a single cell
my_headings = my_headings.groupby('heading')['raw_description'].agg(lambda x: ' '.join(x)).reset_index()
# Rename the description
my_headings.rename(columns={'raw_description':'merged_description'}, inplace=True)
# Add additional columns for curated description and mappings
my_headings['curated_description'] = ''
my_headings['mapped_to'] = '' # It may be the case that some headings in the list did not exist in the PDF, so the must be mapped to another headings that do exist.
# Show count of rows to curate
print('Rows to curate: '+str(len(my_headings)))
my_headings

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\sergio\anaconda3\envs\thesis\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\sergio\anaconda3\envs\thesis\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\sergio\anaconda3\envs\thesis\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\sergio\anaconda3\envs\thesis\lib\site-packages\py4j\clientserver.py", line 503, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [14]:
# Export to CSV for manual resolution of descriptions
if (len(my_headings)>0):
    file_name = './hs_curated_'+str(int(time.time()*1000))+'.csv'
    my_headings.to_csv(file_name, index=None)
    print("Proceed to manual curation process")
else:
    print("No new headings to curate")

No new headings to curate
